In [ ]:
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 52.3 MB/s eta 0:00:00


In [ ]:
import torch
from speechbrain.inference.TTS import FastSpeech2
from speechbrain.inference.vocoders import HIFIGAN
from IPython.display import Audio
import torchaudio

# Load FastSpeech2 model
print("Loading FastSpeech2 model...")
fastspeech2 = FastSpeech2.from_hparams(source="speechbrain/tts-fastspeech2-ljspeech")

print("Loading HiFiGAN vocoder...")
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech")

# Phoneme input for "How are you" - using SpeechBrain's phoneme format
#input_phonemes = ['HH', 'AW', 'AA', 'R', 'Y', 'UW']
#input_phonemes = ['W', 'AH', 'T', 'IH', 'Z', 'Y', 'UH', 'R', 'N', 'EY', 'M']
input_phonemes=['M', 'AY', 'N', 'EY', 'M', 'IH', 'Z']

print(f"Converting phonemes: {input_phonemes}")

# Generate mel-spectrogram from phonemes using FastSpeech2 (correct method!)
mel_output, durations, pitch, energy = fastspeech2.encode_phoneme(
    [input_phonemes],
    pace=1,          # scale up/down the speed
    pitch_rate=1,    # scale up/down the pitch
    energy_rate=1,   # scale up/down the energy
)

# Generate waveform from mel-spectrogram using HiFiGAN
waveform = hifi_gan.decode_batch(mel_output)

# Save and play audio
output_path = "fastspeech2_phoneme_output.wav"
torchaudio.save(output_path, waveform.squeeze(1), 22050)

print("Audio generated with FastSpeech2 using direct phoneme input! Playing...")
Audio(output_path)







INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/tts-fastspeech2-ljspeech' if not cached


Loading FastSpeech2 model...


INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/tts-fastspeech2-ljspeech' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Fetching files for pretraining (no collection directory set)
INFO:speechbrain.utils.fetching:Fetch spn_predictor.ckpt: Fetching from HuggingFace Hub 'speechbrain/tts-fastspeech2-ljspeech' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["spn_predictor"] = /root/.cache/huggingface/hub/models--speechbrain--tts-fastspeech2-ljspeech/snapshots/c50d854a2b3f6be0e08b694c67a92ca56228de18/spn_predictor.ckpt
INFO:speechbrain.utils.fetching:Fetch model.ckpt: Fetching from HuggingFace Hub 'speechbrain/tts-fastspeech2-ljspeech' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["model"] = /root/.cache/huggingface/hub/models--speechbrain--tts-fastspeech2-ljspeech/snapshots/c50d854a2b3f6be0e08b694c67a92ca56228de18/model.ckpt
INFO:speechbrain.utils.parameter_t

Loading HiFiGAN vocoder...


INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/tts-hifigan-ljspeech' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Fetching files for pretraining (no collection directory set)
INFO:speechbrain.utils.fetching:Fetch generator.ckpt: Fetching from HuggingFace Hub 'speechbrain/tts-hifigan-ljspeech' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["generator"] = /root/.cache/huggingface/hub/models--speechbrain--tts-hifigan-ljspeech/snapshots/17fbdc3aae35b81e1554111fa54eab5f2b70cedb/generator.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: generator
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): generator -> /root/.cache/huggingface/hub/models--speechbrain--tts-hifigan-ljspeech/snapshots/17fbdc3aae35b81e1554111fa54eab5f2b70cedb/generator.ckpt


Converting phonemes: ['M', 'AY', 'N', 'EY', 'M', 'IH', 'Z']
Audio generated with FastSpeech2 using direct phoneme input! Playing...


In [ ]:
import torch
from speechbrain.inference.TTS import FastSpeech2
from speechbrain.inference.vocoders import HIFIGAN
from IPython.display import Audio
import torchaudio
import time

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load FastSpeech2
fastspeech2 = FastSpeech2.from_hparams(source="speechbrain/tts-fastspeech2-ljspeech")

# Load HiFiGAN
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech")

# Move vocoder to GPU
if torch.cuda.is_available():
    hifi_gan.mods.generator.to(device)
    print("✅ HiFi-GAN vocoder moved to GPU")

# Input phonemes
input_phonemes = ['W', 'AH', 'T', 'IH', 'Z', 'Y', 'UH', 'R', 'N', 'EY', 'M']


print(f"Converting phonemes: {input_phonemes}")

start = time.time()

# Generate mel spectrogram (CPU-safe)
mel_output, durations, pitch, energy = fastspeech2.encode_phoneme(
    [input_phonemes],
    pace=1.0,
    pitch_rate=1.0,
    energy_rate=1.0,
)

# Move mel to GPU
if torch.cuda.is_available():
    mel_output = mel_output.to(device)

# 🔥 Decode directly via generator (avoids CPU rebuild)
with torch.no_grad():
    waveform = hifi_gan.mods.generator(mel_output)

end = time.time()
print(f"Inference completed in {end - start:.2f} seconds")

# Save & play
output_path = "fastspeech2_gpu_vocoder.wav"
torchaudio.save(output_path, waveform.squeeze(1).cpu(), 22050)

print("HiFi-GAN device:", next(hifi_gan.mods.generator.parameters()).device)
Audio(output_path)


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/tts-fastspeech2-ljspeech' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/tts-fastspeech2-ljspeech' if not cached


Using device: cuda


DEBUG:speechbrain.utils.parameter_transfer:Fetching files for pretraining (no collection directory set)
INFO:speechbrain.utils.fetching:Fetch spn_predictor.ckpt: Fetching from HuggingFace Hub 'speechbrain/tts-fastspeech2-ljspeech' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["spn_predictor"] = /root/.cache/huggingface/hub/models--speechbrain--tts-fastspeech2-ljspeech/snapshots/c50d854a2b3f6be0e08b694c67a92ca56228de18/spn_predictor.ckpt
INFO:speechbrain.utils.fetching:Fetch model.ckpt: Fetching from HuggingFace Hub 'speechbrain/tts-fastspeech2-ljspeech' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["model"] = /root/.cache/huggingface/hub/models--speechbrain--tts-fastspeech2-ljspeech/snapshots/c50d854a2b3f6be0e08b694c67a92ca56228de18/model.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: spn_predictor, model
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from loca

✅ HiFi-GAN vocoder moved to GPU
Converting phonemes: ['W', 'AH', 'T', 'IH', 'Z', 'Y', 'UH', 'R', 'N', 'EY', 'M']
Inference completed in 1.20 seconds
HiFi-GAN device: cuda:0




> This is deployment code.



In [ ]:
import torch
import torchaudio
from speechbrain.inference.TTS import FastSpeech2
from speechbrain.inference.vocoders import HIFIGAN
import time

# 🔧 Global model cache (so they load once)
_fastspeech2_model = None
_hifigan_model = None
_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_tts_models():
    """Loads and caches the FastSpeech2 + HiFiGAN models."""
    global _fastspeech2_model, _hifigan_model

    if _fastspeech2_model is None:
        print("Loading FastSpeech2 model...")
        _fastspeech2_model = FastSpeech2.from_hparams(source="speechbrain/tts-fastspeech2-ljspeech")

    if _hifigan_model is None:
        print("Loading HiFiGAN vocoder...")
        _hifigan_model = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech")

        if torch.cuda.is_available():
            _hifigan_model.mods.generator.to(_device)
            print("✅ HiFi-GAN vocoder moved to GPU")

    return _fastspeech2_model, _hifigan_model


def phoneme_to_audio(phonemes, pace=1.0, pitch_rate=1.0, energy_rate=1.0, sr=22050):
    """
    Converts a list of ARPAbet phonemes into a speech waveform.

    Args:
        phonemes: list of str (e.g., ['W','AH','T','IH','Z','Y','UH','R','N','EY','M'])
        pace: float - control speech speed
        pitch_rate: float - control pitch
        energy_rate: float - control energy
        sr: int - output sampling rate

    Returns:
        waveform (torch.Tensor), sampling_rate
    """
    fastspeech2, hifigan = load_tts_models()

    # 🕒 Measure inference
    start = time.time()

    # Generate mel-spectrogram (FastSpeech2, CPU-safe)
    mel, durations, pitch, energy = fastspeech2.encode_phoneme(
        [phonemes],
        pace=pace,
        pitch_rate=pitch_rate,
        energy_rate=energy_rate,
    )

    # Move mel to GPU for fast vocoding
    if torch.cuda.is_available():
        mel = mel.to(_device)

    # Generate waveform directly with GPU vocoder
    with torch.no_grad():
        waveform = hifigan.mods.generator(mel)

    elapsed = time.time() - start
    print(f"🕒 Inference time: {elapsed:.2f}s (device: {_device})")

    return waveform.squeeze(1).cpu(), sr
